<img src="docs/images/DSPy8.png" alt="DSPy7 Image" height="150"/>

## **DSPy**: Programming with Foundation Models

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/intro.ipynb)

This notebook introduces the **DSPy** framework for **Programming with Foundation Models**, i.e., language models (LMs) and retrieval models (RMs).

**DSPy** emphasizes programming over prompting. It unifies techniques for **prompting** and **fine-tuning** LMs as well as improving them with **reasoning** and **tool/retrieval augmentation**, all expressed through a _minimalistic set of Pythonic operations that compose and learn_.

**DSPy** provides **composable and declarative modules** for instructing LMs in a familiar Pythonic syntax. On top of that, **DSPy** introduces an **automatic compiler that teaches LMs** how to conduct the declarative steps in your program. The **DSPy compiler** will internally _trace_ your program and then **craft high-quality prompts for large LMs (or train automatic finetunes for small LMs)** to teach them the steps of your task.

### 0] Setting Up

As we'll start to see below, **DSPy** can routinely teach powerful models like `GPT-3.5` and local models like `T5-base` or `Llama2-13b` to be much more reliable at complex tasks. **DSPy** will compile the _same program_ into different few-shot prompts and/or finetunes for each LM.

In [2]:
import sys
import os
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, MIPROv2
from rouge_score import rouge_scorer

In [3]:
model_name = 'gemma2:27b'
#model_name = 'qwen2.5:72b'

In [4]:
# does not work right now
#ollama_port = 11434 
#ollama_url = f"http://localhost:{ollama_port}"
#lm = dspy.LM(model=model_name, api_base=ollama_url)
#dspy.settings.configure(lm=lm)

In [5]:
lm = dspy.OllamaLocal(model=model_name)
dspy.settings.configure(lm=lm)

You can build your own **DSPy programs** for various tasks, e.g., question answering, information extraction, or text-to-SQL.

Whatever the task, the general workflow is:

1. **Collect a little bit of data.** Define examples of the inputs and outputs of your program (e.g., questions and their answers). This could just be a handful of quick examples you wrote down. If large datasets exist, the more the merrier!
1. **Write your program.** Define the modules (i.e., sub-tasks) of your program and the way they should interact together to solve your task.
1. **Define some validation logic.** What makes for a good run of your program? Maybe the answers need to have a certain length or stick to a particular format? Specify the logic that checks that.
1. **Compile!** Ask **DSPy** to _compile_ your program using your data. The compiler will use your data and validation logic to optimize your program (e.g., prompts and modules) so it's efficient and effective!
1. **Iterate.** Repeat the process by improving your data, program, validation, or by using more advanced features of the **DSPy** compiler.

### Data

In [6]:
from datasets import load_dataset

multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20230518")
print(multi_lexsum)

DatasetDict({
    train: Dataset({
        features: ['id', 'sources', 'sources_metadata', 'summary/long', 'summary/short', 'summary/tiny', 'case_metadata'],
        num_rows: 3177
    })
    validation: Dataset({
        features: ['id', 'sources', 'sources_metadata', 'summary/long', 'summary/short', 'summary/tiny', 'case_metadata'],
        num_rows: 454
    })
    test: Dataset({
        features: ['id', 'sources', 'sources_metadata', 'summary/long', 'summary/short', 'summary/tiny', 'case_metadata'],
        num_rows: 908
    })
})


The core data type for data in DSPy is Example. You will use Examples to represent items in your training set and test set. DSPy Examples are similar to Python dicts but have a few useful utilities. Your DSPy modules will return values of the type Prediction, which is a special sub-class of Example. When you use DSPy, you will do a lot of evaluation and optimization runs. Your individual datapoints will be of type Example.

Select 100 train examples and 20 dev examples where all 3 summaries are present

In [7]:
trainset = multi_lexsum['train'].filter(
    lambda x: x['summary/tiny'] is not None and x['summary/short'] is not None and x['summary/long'] is not None
).select(range(100))

devset = multi_lexsum['validation'].filter(
    lambda x: x['summary/tiny'] is not None and x['summary/short'] is not None and x['summary/long'] is not None
).select(range(20))

#### Create Examples

In [8]:
def join_sources(x):
    x['sources'] = ' '.join(x['sources'])
    return x

trainset = trainset.map(join_sources, batched=False)
devset = devset.map(join_sources, batched=False)

In [9]:
# Tell DSPy that the joined 'sources' field is the input. Any other fields are labels and/or metadata
trainset = [dspy.Example(
    doc=x['sources'],
    long=x['summary/long'],
    short=x['summary/short'],
    tiny=x['summary/tiny']
).with_inputs('doc') for x in trainset]

devset = [dspy.Example(
    doc=' '.join(x['sources']),
    long=x['summary/long'],
    short=x['summary/short'],
    tiny=x['summary/tiny']
).with_inputs('doc') for x in devset]

# ['doc', 'long', 'short', 'tiny']
len(trainset), len(devset)

(100, 20)

**DSPy** typically requires very minimal labeling. Whereas your pipeline may involve six or seven complex steps, you only need labels for the initial question and the final answer. **DSPy** will bootstrap any intermediate labels needed to support your pipeline. If you change your pipeline in any way, the data bootstrapped will change accordingly!

Now, let's look at some data examples.

In [10]:
train_example = trainset[1]
print(f"short: {train_example.short}")
print(f"tiny: {train_example.tiny}")

short: Two men who were arrested for trespassing on property of businesses open to the public filed a lawsuit in the U.S. District Court for the Western District of Michigan against the city of Grand Rapids, its chief of police, and two individual officers. The plaintiffs claimed that the Grand Rapids Police Department's policy and practice of arresting individuals for trespass -- without probable cause and based on general Letters of Intent to Prosecute signed by Grand Rapids businesses -- results in unreasonable searches and seizures in violation of the Fourth Amendment. The parties came to a private settlement agreement for damages and attorney's fees in late 2019. The Judge dismissed the case in early 2020.
tiny: Settlement reached in 2019 for @ACLU case on arrests for trespassing without a warning under Grand Rapids, MI's "No Trespass Letters" policy (W.D. Mich.)


In [11]:
dev_example = devset[2]
print(f"short: {dev_example.short}")
print(f"tiny: {dev_example.tiny}")

short: Pretrial detainees file lawsuit against Middlesex County in November 2015 to ameliorate the unconstitutional conditions of solitary confinement in the Middlesex County Jail. In September 2018, the parties reached a settlement agreement that restricted the maximum amount of time allowed in isolation and provides those in isolation with opportunities to interact with others.
tiny: Pretrial detainees settled this class action against Middlesex County to provide 28 hours per week of out-of-cell time and mental health screenings to people held in solitary confinement.


After loading the raw data, we'd applied `with_inputs(' '.join(x['sources']))` to each example to tell **DSPy** that our input field in each example will be just `doc`. Any other fields are labels or metadata that are not given to the system.

### Basic zero shot prompt

tiny ~ 25 words

short ~ 130 words

long ~ 650 words

Generate a {summary_type} summary of maximum {max_tokens[summary_type]} tokens of the following text:

#### Short Summary

##### Signature

In [12]:
class ShortSummSig(dspy.Signature):
    """Generate short summaries of about 130 words."""
    # input
    doc = dspy.InputField()
    # output
    short = dspy.OutputField()

In `ShortSumm`, the docstring describes the sub-task. Each `InputField` or `OutputField` can optionally contain a description `desc` too. When it's not given, it's inferred from the field's name (e.g., `question`).

Notice that there isn't anything special about this signature in **DSPy**. We can just as easily define a signature that takes a long snippet from a PDF and outputs structured information, for instance.

Anyway, now that we have a signature, let's define and use a **Predictor**. A predictor is a module that knows how to use the LM to implement a signature. Importantly, predictors can **learn** to fit their behavior to the task!

```
dspy.Example(field1=value, field2=value2, ...)
```

In [13]:
# Define the predictor.
generate_short = dspy.Predict(ShortSummSig)

# Call the predictor on a particular input
pred = generate_short(doc=dev_example.doc)

# Print the prediction
#print(f"Doc: {dev_example.doc}")
print(f"Generated short: {pred.short}")
print('')
print(f"Ground truth short: {dev_example.short}")

 		You are using the client OllamaLocal, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Generated short: This appears to be a legal document excerpt detailing court activity. Here's a breakdown:

* **Dismissal:** The case (likely a civil lawsuit, given the "cv" designation) was dismissed with prejudice against Middlesex County. This means the plaintiff cannot refile the same claim.
* **Stipulation:**  The dismissal was agreed upon by both parties (the plaintiff and Middlesex County).
* **Dates:** Key dates include:
    * 10/24/2018: Judge Peter G. Sheridan signed the dismissal order.
    * 10/25/2018: The dismissal was officially entered into the court record.
* **PACER Transaction:** This section indicates

Ground truth short: Pretrial detainees file lawsuit against Middlesex County in November 2015 to ameliorate the unconstitutional conditions of solitary confinement in the Middlesex County Jail. In September 2018, the parties reached a settlement agreement that restricted the maximum amount of time allowed in isolation and provides those in isolation with opportunities

#### Long Summary

In [29]:
class LongSummSig(dspy.Signature):
    """Generate long summaries."""
    # input
    doc = dspy.InputField()
    # output
    long = dspy.OutputField()

In [30]:
# Define the predictor.
generate_long = dspy.Predict(LongSummSig)

# Call the predictor on a particular input
pred = generate_long(doc=dev_example.doc)

# Print the prediction
#print(f"Doc: {dev_example.doc}")
print(f"Generated long: {pred.long}")
print('')
print(f"Ground truth long: {dev_example.long}")

Generated long: This text appears to be a legal document receipt from the PACER (Public Access to Court Electronic Records) system. Here's a breakdown of what it likely means:

* **Case Information:** The document refers to a case with docket number "3:15-cv-07920". This suggests a civil case filed in the United States District Court for the District of New Jersey (the "3" indicates the judicial district).
* **Stipulation of Dismissal:**  The text mentions a "STIPULATION of Dismissal with Prejudice as to Defendant, Middlesex County". This means that the plaintiff(s) and defendant(s), including Middlesex County, agreed to dismiss the case permanently. "With prejudice"

Ground truth long: On November 5, 2015, the plaintiffs, nine pretrial detainees, filed this class action in the United States District Court of New Jersey. The plaintiffs sued Middlesex County under 42 U.S.C. § 1983 for the deprivation of rights secured by the Eighth and Fourteenth Amendments to the United States Constitu

#### Evaluation

##### Metric

In [31]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def eval_summ_short(example, pred, trace=None):
    return scorer.score(example.short.lower(), pred.short.lower())['rouge1'][2]

def eval_summ_long(example, pred, trace=None):
    return scorer.score(example.long.lower(), pred.long.lower())['rouge1'][2]

r1_short = eval_summ_short
r1_long = eval_summ_long

##### Program

In [32]:
class ShortSummProg(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_summ = dspy.Predict(ShortSummSig)

    def forward(self, doc):
        return self.generate_summ(doc=doc)

class LongSummProg(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_summ = dspy.Predict(LongSummSig)

    def forward(self, doc):
        return self.generate_summ(doc=doc)
        
shortsumm = ShortSummProg()
longsumm = LongSummProg()

##### Evaluate

In [20]:
evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=0)

In [22]:
evaluator(shortsumm, metric=r1_short)

Average Metric: 4.149412227978533 / 20  (20.7): 100%|█| 20/20 [01:07<00:00,  3


20.75

In [26]:
evaluator(longsumm, metric=r1_long)

Average Metric: 3.0440304603629977 / 20  (15.2): 100%|█| 20/20 [01:24<00:00,  


15.22

### Optimized few-shot with bootstrapped demonstrations

In [33]:
bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    max_bootstrapped_demos=8, #8
    max_labeled_demos=8, #8
    num_candidate_programs=10,
    num_threads=8, #8
    metric=r1_long,
    #teacher_settings=dict(lm=gpt4T)
)

Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 10 candidate sets.


In [34]:
longsumm_fewshot = bootstrap_optimizer.compile(longsumm, trainset=trainset, valset=devset)

Average Metric: 2.9462854710706337 / 20  (14.7): 100%|█| 20/20 [00:34<00:00,  


New best score: 14.73 for seed -3
Scores so far: [14.73]
Best score so far: 14.73


Average Metric: 3.0062489253735536 / 20  (15.0): 100%|█| 20/20 [01:42<00:00,  


New best score: 15.03 for seed -2
Scores so far: [14.73, 15.03]
Best score so far: 15.03


  8%|███▎                                     | 8/100 [01:17<14:56,  9.75s/it]


...

Scores so far: [14.73, 15.03, 14.7, 14.58, 14.92, 15.03, 14.81, 14.8, 14.82, 14.98, 14.78, 14.82]
Best score so far: 15.03


  8%|███▎                                     | 8/100 [01:18<14:58,  9.76s/it]


Bootstrapped 8 full traces after 8 examples for up to 1 rounds, amounting to 8 attempts.


Average Metric: 2.979916205066145 / 20  (14.9): 100%|█| 20/20 [01:44<00:00,  5

Scores so far: [14.73, 15.03, 14.7, 14.58, 14.92, 15.03, 14.81, 14.8, 14.82, 14.98, 14.78, 14.82, 14.9]
Best score so far: 15.03
13 candidate programs found.


In [18]:
# short
#max_bootstrapped_demos=4
#max_labeled_demos=4
21.16

#max_bootstrapped_demos=8
#max_labeled_demos=8
23.4

# long


### MIPROv2

In [ ]:
# Initialize optimizer
teleprompter = MIPROv2(
    metric=r1_long,
    #auto="light", # Can choose between light, medium, and heavy optimization runs
)

# Optimize program
print(f"Optimizing program with MIPRO...")
optimized_program = teleprompter.compile(
    longsumm.deepcopy(),
    trainset=trainset,
    valset=devset,
    minibatch_size=5,
    max_bootstrapped_demos=3,
    max_labeled_demos=4,
    requires_permission_to_run=False,
)

Optimizing program with MIPRO...

==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
These will be used as few-shot example candidates for our program and for creating instructions.

Bootstrapping N=10 sets of demonstrations...
Bootstrapping set 1/10
Bootstrapping set 2/10
Bootstrapping set 3/10


  3%|█▏                                       | 3/100 [00:15<08:20,  5.16s/it]


Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.
Bootstrapping set 4/10


  3%|█▏                                       | 3/100 [00:25<13:34,  8.40s/it]


Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.
Bootstrapping set 5/10


...

Average Metric: 0.7378720097512407 / 5  (14.8): 100%|█| 5/5 [00:36<00:00,  7.3


Score: 14.76 on minibatch of size 5 with parameters ['Predictor 1: Instruction 0', 'Predictor 1: Few-Shot Set 7'].
Minibatch scores so far: [20.89, 12.25, 18.59, 13.01, 13.01, 15.9, 16.7, 15.12, 14.76]
Full eval scores so far: [15.1]
Best full score so far: 15.1


== Minibatch Trial 10 / 30 ==


Average Metric: 0.6944680231977017 / 5  (13.9): 100%|█| 5/5 [00:37<00:00,  7.5


Score: 13.89 on minibatch of size 5 with parameters ['Predictor 1: Instruction 9', 'Predictor 1: Few-Shot Set 7'].
Minibatch scores so far: [20.89, 12.25, 18.59, 13.01, 13.01, 15.9, 16.7, 15.12, 14.76, 13.89]
Full eval scores so far: [15.1]
Best full score so far: 15.1


===== Full Eval 1 =====
Doing full eval on next top averaging program (Avg Score: 20.89) from minibatch trials...


Average Metric: 2.5339986207297494 / 16  (15.8):  80%|▊| 16/20 [00:50<00:08,  

In [ ]:
# Save optimize program for future use
#optimized_program.save(f"mipro_optimized")